In [ ]:
# Now let's test the connection
from ai21 import AI21Client
import os

# paste your API key here or set it as an environment variable
API_KEY = os.environ.get("AI21_API_KEY")

client = AI21Client(api_key=API_KEY)

Let's first upload the attached document to the file library

In [ ]:
client.library.files.create("SYSTEM AIR CONDITIONER.pdf")

You can check out the File Library in Studio to make sure the file is there. You can also add more files

Now let's run the following question using Maestro. Note that we will be using Requirements as well in order to get the output we want.
Note that we are introducing a new parameters:
1. `tools` - The list of tools Maestro will have in its disposal to fulfill the task
2. `include` was updated to return the "data_sources" - This will make the Run result also contain the Data Sources that were used in order to generate the answer

Note:
The `budget` field, in addition to the validated output efforts will also affect how Maestro works here in several ways:
1. More compute resources will be invested in the task fulfillment strategy, e.g. more steps may run
2. More retrieval strategies may be employed in order to get a more relevant context

In [ ]:
input = """
1. After power-up, the display shows Ch and won’t allow operation. Explain the compressor preheating logic (2.5-hour heat, first-install requires 6 hours of power applied), the condition for Ch to clear, and what is or isn’t safe to bypass. Provide a customer-friendly ETA message

2. You are an AI assistant helping explain why the display shows Ch after power-up.
Using the attached manual, describe the compressor preheating logic, how long it lasts, and when operation becomes available.
Include safe advice for the user while waiting. Add a simple “Do / Don’t” list for safe handling.
"""

requirements = [
    {
        "name": "File grounding",
        "description": "Use data from the attached manual; quote code labels and thresholds",
    },
    {
        "name": "Cause prioritization",
        "description": "If multiple causes are possible, order them by likelihood and cross-check against sensor readings, phase conditions, or other relevant data in the file",
    },
    {
        "name": "Safety tagging",
        "description": "Flag and clearly label actions restricted to qualified technicians",
    },
    {
        "name": "Two-layer output",
        "description": "End each response with a plain-language customer summary followed by a detailed technical checklist for field technicians",
    },
]

run = client.beta.maestro.runs.create_and_poll(
    input=input,
    requirements=requirements,
    budget="medium",
    tools=[{"type": "file_search"}],
    include=[
        "requirements_result",
        "data_sources",
    ],
)

print(run.id)
print(run.result)
print(run.requirements_result)
print(run.data_sources)
